In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# [speed&direction] (캐글 따라하기) 타이타닉 데이터 분석

이번 블로그는 다음의 참고자료를 통해 재구성한 것이다.

- 캐글 코리아 블로그(타이타닉 튜토리얼) : https://kaggle-kr.tistory.com/17#2_6
- [수비니움의 캐글 따라하기] 타이타닉 : Beginner Ver. : https://www.kaggle.com/subinium/subinium-tutorial-titanic-beginner/data?select=train.csv
- 인문계공돌이님 : https://bizzengine.tistory.com/178

- 데이터 분석을 공부하는 초보자로써 캐글에서 유명한 타이타닉 데이터 분석을 하려고 한다.
- 타이타닉은 역사상 최악의 해난 사고로 영화로도 그 스토리가 제작이 되었다.
- 이번 블로그에서 타이타닉에 탑승한 사람들의 여러가지 피처를 활용하여, 승선한 사람들의 생존여부를 예측하는 모델을 생성할 것이다.
- 1912년 당시에는 계급이 있었고, 영화에서도 나왔지만 타이타닉의 선실은 1.2.3 등급의 선실로 나뉘어져 있었으며 구명 보트에 사람을 태울때도 1등급의 선실에 있는 사람을 먼저 태운 후 여자와 아이들을 태웠다. 캐글의 데이터도 이와 같은지 분석을 해보려고 한다.

#### 시작에 앞서

- 데이터 정보를 최소한으로 살피면서 분석한다.
- 어려운 메서드나 복잡한 함수는 최대한 피하면서 분석한다.
- 분석을 위한 필수적인 요소와 순서를 서술한다.
- 초보자가 쉽게 따라할수 있게 한다.

#### 1. 라이브러리 불러오기


타이타닉 데이터 분석에 앞서 필요한 라이브러리를 불러오자.

In [ ]:
# 데이터 분석에 관련한 라이브러리 불러오기.
import pandas as pd
from pandas import Series, DataFrame
import numpy as np

In [ ]:
# 데이터 시각화 관련한 라이브러리 불러오기.
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid') 
%matplotlib inline 

In [ ]:
# Scikit-Learn의 다양한 머신러닝 모듈 불러오기.
# 분류 알고리즘 중에서 LogisticRegression, LinearSVC, RandomForestClassifier, KNeighborsClassifier 알고리즘을 사용해 보자.
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

#### 2. 데이터 가져오기
캐글 뿐만 아니라 데이터 분석에서 가장 많이 사용되는 파일 형식은 csv 파일이다. 코드로 데이터를 읽는 방법은 여러가지 있지만, 그 중에서도 가장 유용한 것은 pd.read_csv로 읽는 방법이다.

In [ ]:
# 데이터 가져오기
train_df = pd.read_csv("../input/titanic-machine-learning-from-disaster/train.csv")
test_df = pd.read_csv("../input/titanic-machine-learning-from-disaster/test.csv")

In [ ]:
# head() 메서드를 이용해 앞의 5번째까지의 데이터 확인하기
train_df.head()

위의 결과를 보면 앞의 번호는 큰 의미를 가지지 않는다. 이름과 티켓의 경우에는 복잡하기 때문에 처리하기 어려워 보인다. 데이터의 정보는 info 메서드로 확인할 수 있다. train 데이터와 test 데이터를 확인해보자.

In [ ]:
train_df.info()
print('-'*20)
test_df.info()

위 결과를 보면 각각의 데이터 개수는 891개, 418개 이고, column은 train 데이터는 12개, test 데이터는 11개이다. train 데이터가 12개인 이유는 survived 즉, 생존 여부도 train 데이터는 알고 있기 때문이다.

#### 주의해야할 점

- 결측값을 drop할지 아니면 default 값으로 채워넣을 것인가
- cabin, age, embarked 피처



데이터 분석을 위해 필요없다고 생각되는 부분을 지우자. 우리는 복잡한 형태의 PassengerID와 Name, Ticket을 지울 것이다. 사실 이름과 티켓에서 가져올 수 있는 데이터는 없기 때문이다. 하지만 이번 분석에서 결과물은 'PassengerID, 'Survived' 피처가 필요하기 때문에 train 데이터에서만 지우자.

In [ ]:
train_df = train_df.drop(['PassengerId', 'Name', 'Ticket'], axis=1)
test_df = test_df.drop(['Name','Ticket'], axis=1)

#### 3. 각각의 데이터 처리하기

남은 데이터의 종류는 아래와 같다.

- Pclass
- Sex
- Age
- SibSp
- Parch
- Fare
- Cabin
- Embarked

##### 3.1 Pclass

Pclass는 서수형 데이터이다. 선실의 1,2,3 등급을 나타낸다. 위에서 확인했을 때 결측값이 없다고 나왔다.
데이터 확인과 데이터를 변환해보자. 우선 각 unique한 value에 대한 카운팅은 value_counts() 메서드로 확인할 수 있다.

In [ ]:
train_df['Pclass'].value_counts()

1,2,3은 정수이기 때문에 실수로 바꾸면 되지 않을까 생각할 수 있다. 하지만 1,2,3 등급은 경우에 따라 다를 수 있지만 연속적인 데이터가 아니다. 그리고 각각의 차이 또한 균등하지 않다. 그렇기 때문에 범주형 데이터로 인식하고 인코딩 해야 한다.
이 데이터는 범주형 데이터이기 때문에 one-hot-encoding을 pd.get_dummies() 메서드로 인코딩하자.

In [ ]:
pclass_train_dummies = pd.get_dummies(train_df['Pclass'])
pclass_test_dummies = pd.get_dummies(test_df['Pclass'])

train_df.drop(['Pclass'], axis=1, inplace=True)
test_df.drop(['Pclass'], axis=1, inplace=True)

train_df = train_df.join(pclass_train_dummies)
test_df = test_df.join(pclass_test_dummies)

##### 3.2 Sex

Sex는 성별이다. 남과 여로 나뉘기 때문에 이 또한 one-hot-encoding을 하자.


In [ ]:
sex_train_dummies = pd.get_dummies(train_df['Sex'])
sex_test_dummies = pd.get_dummies(test_df['Sex'])

sex_train_dummies.columns = ['Female', 'Male']
sex_test_dummies.columns = ['Female', 'Male']

train_df.drop(['Sex'], axis=1, inplace=True)
test_df.drop(['Sex'], axis=1, inplace=True)

train_df = train_df.join(sex_train_dummies)
test_df = test_df.join(sex_test_dummies)

##### 3.3 Age

Age는 나이이다. 나이는 연속형 데이터이기 때문에 큰 처리가 필요없다. 하지만 일부 결측값이 존재한다. 이를 채울 수 있는 방법은 아래 4가지가 있다.

- 랜덤값
- 평균값
- 중간값
- drop

이번에는 우선 평균값으로 대체하자. 데이터의 통일성을 가지기 위해 train 데이터셋의 평균값으로 train,test 데이터셋을 채워보자.

In [ ]:
train_df["Age"].fillna(train_df["Age"].mean() , inplace=True)
test_df["Age"].fillna(train_df["Age"].mean() , inplace=True)

##### 3.4 Sibsp & Panch

형제 자매와 부모님은 가족으로 함께 처리할 수 있다. 하지만 바꿀 필요는 없다.

##### 3.5 Fare 

Fare는 탑승료이다. test 데이터셋에 1개의 데이터가 이상하게 비어있다. 이 부분은 fillna 메서드로 채울 것이다. 무임 승선이라 생각하고 0으로 입력할 것이다.

In [ ]:
test_df["Fare"].fillna(0, inplace=True)

##### 3.6 cabin

Cabin은 객실을 뜻한다. 결측값이 너무 많기 때문에 drop하자.

In [ ]:
train_df = train_df.drop(['Cabin'], axis=1)
test_df = test_df.drop(['Cabin'], axis=1)

##### 3.7 Embarked

Emabarked는 탑승하는 항구를 뜻한다. 우선 데이터를 확인해 보자.

In [ ]:
train_df['Embarked'].value_counts()

In [ ]:
test_df['Embarked'].value_counts()

위 결과 S가 대다수이고 일부 결측값이 존재한다. 빈 부분은 S로 채우고 시작하자.

In [ ]:
train_df['Embarked'].fillna('S', inplace=True)
test_df['Embarked'].fillna('S', inplace=True)

In [ ]:
embarked_train_dummies = pd.get_dummies(train_df['Embarked'])
embarked_test_dummies = pd.get_dummies(test_df['Embarked'])

embarked_train_dummies.columns = ['S', 'C', 'Q']
embarked_test_dummies.columns = ['S', 'C', 'Q']

train_df.drop(['Embarked'], axis=1, inplace=True)
test_df.drop(['Embarked'], axis=1, inplace=True)

train_df = train_df.join(embarked_train_dummies)
test_df = test_df.join(embarked_test_dummies)

#### 4. 데이터 나누기

이제 학습용 데이터를 위해 데이터를 나누어야 한다.
Passenger Id와 Survived 형태로 아래와 같이 데이터를 나눠준다.

In [ ]:
X_train = train_df.drop("Survived",axis=1)
Y_train = train_df["Survived"]
X_test = test_df.drop("PassengerId",axis=1).copy()

#### 5. 머신러닝 알고리즘 적용하기


이제 LogisticRegression, LinearSVC, RandomForestClassifier, KNeighborsClassifier 알고리즘을 각각 적용해 보자.

In [ ]:
# Logistic Regression

logreg = LogisticRegression()
logreg.fit(X_train, Y_train)

Y_pred = logreg.predict(X_test)

logreg.score(X_train, Y_train)

In [ ]:
# SVC

svc = SVC()

svc.fit(X_train, Y_train)

Y_pred = svc.predict(X_test)

svc.score(X_train, Y_train)

In [ ]:
# Random Forests

random_forest = RandomForestClassifier(n_estimators=100)

random_forest.fit(X_train, Y_train)

Y_pred = random_forest.predict(X_test)

random_forest.score(X_train, Y_train)

In [ ]:
# KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors = 3)

knn.fit(X_train, Y_train)

Y_pred = knn.predict(X_test)

knn.score(X_train, Y_train)

랜덤 포레스트가 가장 좋은 결과를 내는 것을 알 수 있다.

#### 7. 제출용 파일 만들기

In [ ]:
# Random Forests

random_forest = RandomForestClassifier(n_estimators=100)
random_forest.fit(X_train, Y_train)
Y_pred = random_forest.predict(X_test)
random_forest.score(X_train, Y_train)

submission = pd.DataFrame({
        "PassengerId": test_df["PassengerId"],
        "Survived": Y_pred
    })
submission.to_csv('titanic.csv', index=False)